<a href="https://colab.research.google.com/github/Akshitwadhwa/Stock-modelling/blob/main/Notebook_4_Stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This is the second colab notebook made for selecting the stocks using MPT and making optimal portfolios based upon them


### 1.1 In this notebook we have to load the nifty 50 stock list again and collect that data for the same time interval which is from 2010 to 2024


In [33]:
# #### *1. Collect Data*
# •⁠  ⁠Historical price data of 50 stocks (daily/weekly/monthly returns)
# •⁠  ⁠Time horizon: 2010 to 2024


# Importing all the neccesray import numpy as np
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import cvxpy as cp
import random





In [34]:
# We have to take the stocks from the nifty fifty for the india sector so we need to take the yfinance library.


nifty50_Stocks = [
    "RELIANCE.NS", "TCS.NS", "INFY.NS", "HDFCBANK.NS", "ICICIBANK.NS", "KOTAKBANK.NS",
    "HINDUNILVR.NS", "SBIN.NS", "LT.NS", "ITC.NS", "AXISBANK.NS", "BAJFINANCE.NS",
    "ASIANPAINT.NS", "MARUTI.NS", "SUNPHARMA.NS", "ULTRACEMCO.NS", "NESTLEIND.NS", "WIPRO.NS",
    "NTPC.NS", "POWERGRID.NS", "COALINDIA.NS", "TITAN.NS", "TECHM.NS", "GRASIM.NS",
    "BHARTIARTL.NS", "HCLTECH.NS", "DRREDDY.NS", "ADANIENT.NS", "BAJAJFINSV.NS", "TATASTEEL.NS",
    "HDFCLIFE.NS", "SBILIFE.NS", "HEROMOTOCO.NS", "DIVISLAB.NS", "INDUSINDBK.NS", "JSWSTEEL.NS",
    "CIPLA.NS", "EICHERMOT.NS", "UPL.NS", "BPCL.NS", "BRITANNIA.NS", "SHREECEM.NS",
    "ONGC.NS", "APOLLOHOSP.NS", "BAJAJ-AUTO.NS", "M&M.NS", "HINDALCO.NS", "TATACONSUM.NS",
    "TATAMOTORS.NS", "SBICARD.NS"
]
#importing the nifty fifty stocks



In [35]:
# defining the start and the end time for which we need to read the stocks for

price_data = yf.download(nifty50_Stocks, start="2010-04-01", end="2024-04-01")
price_data = price_data.dropna(axis=1, thresh=len(price_data)*0.9)
# which is from 2010 to 2024

# here we have downloaded the data and also reomved the values which are not there in the given time

supposed_return = price_data.pct_change().dropna()




[*********************100%***********************]  50 of 50 completed


### 1.2 What is the mean return for a stock vs a mean return of a portfolio.
--The mean return of stock is the average return of a single stock over time

--It can also be called as how much a stock returns on a monthly basis.
### 1.3 A mean return of a portfolio  can be calculated using the mean return on an investment given the historical returns or the probable rates of returns under giving return scenarios.
 -- The tool given and used by Investors to weigh investment decsions is known as mean varience analysis.

 -- The Varience shows how spread out the returns of a specific security on a daily or weekly basis.

 -- The expected return is a probability expressing the estimated return of the investment in the security.


## **The optimal portfolio will have high return and low variance**

## 2 Variance is a number that represents how varied or spread out the numbers are in a set. For example, variance may tell how spread out the returns of a specific security are on a daily or weekly basis.

In [36]:
mean_returns = supposed_return.mean()
variances = supposed_return.var()


In [37]:
score = mean_returns / variances

score


Price   Ticker       
Close   ADANIENT.NS      1.447990
        APOLLOHOSP.NS    2.291628
        ASIANPAINT.NS    3.424736
        AXISBANK.NS      1.319229
        BAJAJ-AUTO.NS    2.944119
                           ...   
Volume  TECHM.NS              NaN
        TITAN.NS              NaN
        ULTRACEMCO.NS         NaN
        UPL.NS                NaN
        WIPRO.NS              NaN
Length: 235, dtype: float64

### 2.1 After finding out the score of the all the nifty fifty , we see the top two score


In [38]:
top_2_stocks = score.sort_values(ascending=False).head(2).index.tolist()
print(top_2_stocks)



[('Close', 'BRITANNIA.NS'), ('High', 'HINDUNILVR.NS')]


###2.2 Here we have identified the top 2 stocks which have the highest score
-- these are going to be constant in all the portfolios

-- We are then going to make the combination of the portfolios


In [39]:
portfolios = []
for i in range(4):
    # Randomly pick 4 stocks from remaining stocks (excluding the fixed 2)
    random_stocks = random.sample(
        [s for s in supposed_return.columns if s not in top_2_stocks], 4
    )

    # Add the fixed 2 stocks to the random 4 stocks
    portfolio = top_2_stocks + random_stocks

    # Save the portfolio
    portfolios.append(portfolio)




In [ ]:
for idx, pf in enumerate(portfolios):
    print(f"Portfolio {idx+1}: {pf}")

Portfolio 1: [('Close', 'BRITANNIA.NS'), ('High', 'HINDUNILVR.NS'), ('Low', 'BHARTIARTL.NS'), ('Volume', 'INDUSINDBK.NS'), ('High', 'HINDALCO.NS'), ('Low', 'HDFCBANK.NS')]
Portfolio 2: [('Close', 'BRITANNIA.NS'), ('High', 'HINDUNILVR.NS'), ('High', 'ONGC.NS'), ('Volume', 'HDFCBANK.NS'), ('Low', 'ICICIBANK.NS'), ('Low', 'GRASIM.NS')]
Portfolio 3: [('Close', 'BRITANNIA.NS'), ('High', 'HINDUNILVR.NS'), ('Close', 'EICHERMOT.NS'), ('High', 'EICHERMOT.NS'), ('Close', 'KOTAKBANK.NS'), ('Open', 'BPCL.NS')]
Portfolio 4: [('Close', 'BRITANNIA.NS'), ('High', 'HINDUNILVR.NS'), ('Low', 'ITC.NS'), ('Low', 'HINDUNILVR.NS'), ('Low', 'BAJFINANCE.NS'), ('High', 'LT.NS')]


## 3. Now we have to calculate the mean returns of all the portfolios.

In [65]:
portfolio_returns = []
for idx, portfolio in enumerate(portfolios):
    # Get mean return for selected stocks
    sub_returns = supposed_return[portfolio]

    # Equal weights for each stock
    weights = np.array([1/6] * 6)

    # Mean return for the portfolio
    mean_portfolio_return = np.dot(weights, sub_returns.mean())


# weights = np.array([1/6] * 6)

# for i, pf in enumerate(portfolios, 1):
#     pf_returns = supposed_return[pf].dot(weights)
#     cum_returns = (1 + pf_returns).cumprod()
#     cumulative_returns[f'Portfolio {i}'] = cum_returns

mean_portfolio_return

np.float64(inf)

In [61]:
mean_returns_df = pd.DataFrame(portfolio_returns)

portfolio_returns.append({
        'Portfolio': f'Portfolio {idx + 1}',
        'Stocks': portfolio,
        'Mean Return': mean_portfolio_return

    })

### 3.1 After finding out the pairs of the returns , we need to find the covarience matrix of each portfolio


In [62]:

print("📐 Covariance Matrices for Each Portfolio:\n")

for i, pf in enumerate(portfolios, 1):
    cov_matrix = supposed_return[pf].cov()
    print(f"\n📊 Portfolio {i} Covariance Matrix:")
    print(cov_matrix.round(6))

📐 Covariance Matrices for Each Portfolio:


📊 Portfolio 1 Covariance Matrix:
Price                       Close          High     Volume          Low  \
Ticker               BRITANNIA.NS HINDUNILVR.NS HCLTECH.NS NESTLEIND.NS   
Price  Ticker                                                             
Close  BRITANNIA.NS      0.000256      0.000059        NaN     0.000051   
High   HINDUNILVR.NS     0.000059      0.000182        NaN     0.000046   
Volume HCLTECH.NS             NaN           NaN        NaN          NaN   
Low    NESTLEIND.NS      0.000051      0.000046        NaN     0.000194   
       UPL.NS            0.000069      0.000030        NaN     0.000069   
Close  HEROMOTOCO.NS     0.000069      0.000036        NaN     0.000031   

Price                                  Close  
Ticker                  UPL.NS HEROMOTOCO.NS  
Price  Ticker                                 
Close  BRITANNIA.NS   0.000069      0.000069  
High   HINDUNILVR.NS  0.000030      0.000036  
Volume HCLTE

/usr/local/lib/python3.11/dist-packages/numpy/lib/_function_base_impl.py:2767: RuntimeWarning: invalid value encountered in subtract
  X -= avg[:, None]
/usr/local/lib/python3.11/dist-packages/numpy/lib/_function_base_impl.py:2767: RuntimeWarning: invalid value encountered in subtract
  X -= avg[:, None]
/usr/local/lib/python3.11/dist-packages/numpy/lib/_function_base_impl.py:2767: RuntimeWarning: invalid value encountered in subtract
  X -= avg[:, None]


###3.2 Reading this covarince matrix

--**The Matrix can be read as the diagonal as well as the off diagonal**

-- **On one side we have the returns of each of the stocks with regards to each other and then the returns of the stocks with realtion of the other**

-- **If the value is in negative then that means the movement of the returns is in the opposite direction**

In [74]:
risk_free_rate = 0.0001

for i, pf in enumerate(portfolios, 1):
    pf_returns = supposed_return[pf].mean()
    weights = np.array([1/6] * 6)

    expected_return = np.dot(weights, pf_returns)
    cov_matrix = supposed_return[pf].cov().values
    variance = np.dot(weights.T, np.dot(cov_matrix, weights))
    std_dev = np.sqrt(variance)
    sharpe_ratio = (expected_return - risk_free_rate) / std_dev

    print(f"Portfolio {i}:")
    print(f"  Expected Return  = {expected_return:.6f}")
    print(f"  Std Dev (Risk)   = {std_dev:.6f}")
    print(f"  Sharpe Ratio     = {sharpe_ratio:.4f}\n")


Portfolio 1:
  Expected Return  = inf
  Std Dev (Risk)   = nan
  Sharpe Ratio     = nan

Portfolio 2:
  Expected Return  = 0.000810
  Std Dev (Risk)   = 0.010202
  Sharpe Ratio     = 0.0696

Portfolio 3:
  Expected Return  = inf
  Std Dev (Risk)   = nan
  Sharpe Ratio     = nan

Portfolio 4:
  Expected Return  = inf
  Std Dev (Risk)   = nan
  Sharpe Ratio     = nan



/usr/local/lib/python3.11/dist-packages/numpy/lib/_function_base_impl.py:2767: RuntimeWarning: invalid value encountered in subtract
  X -= avg[:, None]
/usr/local/lib/python3.11/dist-packages/numpy/lib/_function_base_impl.py:2767: RuntimeWarning: invalid value encountered in subtract
  X -= avg[:, None]
/usr/local/lib/python3.11/dist-packages/numpy/lib/_function_base_impl.py:2767: RuntimeWarning: invalid value encountered in subtract
  X -= avg[:, None]



### 🔧 Step-by-Step Process:

#### *1. Collect Data*
•⁠  ⁠Historical price data of 50 stocks (daily/weekly/monthly returns)
•⁠  ⁠Time horizon: at least 1–3 years is common

#### *2. Calculate Returns and Covariance*
•⁠  ⁠Compute *mean returns* of each stock
•⁠  ⁠Compute *covariance matrix* of returns (shows how the returns of the stocks move together)

#### *3. Generate All Combinations (Optional for small k)*
•⁠  ⁠If you want exactly *4 stocks*, generate all combinations of 4 stocks from the 50:  
  \[
  \binom{50}{4} = 230300
  \]
  This is computationally heavy, but possible with code (Python/R).

#### *4. For Each 4-Stock Combination:*
•⁠  ⁠Use *quadratic programming* to find the *efficient frontier* and *optimal weights* for the 4 assets
•⁠  ⁠Calculate:
  - *Expected return* of the portfolio
  - *Portfolio variance (risk)*
  - *Sharpe Ratio* (if risk-free rate is available)

#### *5. Select the Best Combination*
•⁠  ⁠Choose the 4-stock portfolio that has:
  - Highest Sharpe Ratio, or
  - Minimum risk for desired return, or
  - Maximum return for a given level of risk

---

